In [1]:
import pandas as pd
import numpy as np
import scipy as sc

import sklearn as sk
from sklearn.model_selection import KFold
 

# Чтение базы фильмов

In [2]:
%%time
movie_titles = pd.read_csv(r'D:\Bugaychenko\Netflix\movie_titles.csv', 
                           encoding = 'ISO-8859-1', 
                           header = None, 
                           names = ['Id', 'Year', 'Name']).set_index('Id')

movie_titles = movie_titles.drop(movie_titles[movie_titles.Year.isnull()].index)
movie_titles.Year = movie_titles.Year.astype(int)

print(movie_titles)

       Year                                               Name
Id                                                            
1      2003                                    Dinosaur Planet
2      2004                         Isle of Man TT 2004 Review
3      1997                                          Character
4      1994                       Paula Abdul's Get Up & Dance
5      2004                           The Rise and Fall of ECW
6      1997                                               Sick
7      1992                                              8 Man
8      2004                         What the #$*! Do We Know!?
9      1991                           Class of Nuke 'Em High 2
10     2001                                            Fighter
11     1999                     Full Frame: Documentary Shorts
12     1947                               My Favorite Brunette
13     2003  Lord of the Rings: The Return of the King: Ext...
14     1982                                 Nature: Ant

# Чтение юзер даты

In [5]:
%%time
df_raw = pd.read_csv(r'D:\Bugaychenko\Netflix\combined_data_1.txt', header=None, 
                                                                        names=['User', 'Rating', 'Date'], 
                                                                        usecols=[0, 1, 2])

index_films = df_raw[(df_raw.Date.isnull())& (df_raw.Rating.isnull())].index # промежутки с оценками на фильмы


# создание столбца №фильма
a = np.empty(len(df_raw), dtype=int)
k =1
for current, next in zip(index_films, index_films[1:]):
    a[current:next] = k
    k+=1
    a[next:] = k
df_raw["Movie"] = a


df = df_raw.drop(index_films) #удаляем пустые строки
df.Rating = df.Rating.astype(int)
df = df[df.Movie.isin(movie_titles.index.unique().values)] # проверка присутствия фильмов

y = df.Rating.values # таргет
np.save('target', y) 

del df_raw
del index_films
del a
del y

#### убрать при пересоздании спарсы
#del df
#del movie_titles

Wall time: 30.1 s


# Создание спарс матрицы



In [8]:
col1 = df.Movie.unique().shape[0]
col2 = df.User.unique().shape[0]
sparse_dataset = sc.sparse.lil_matrix((df.shape[0], col1 + col2))

In [9]:
%%time
customer = {j:i for i, j in enumerate(df.User.unique())}
movie = {j:i for i, j in enumerate(df.Movie.unique())}

for i, j in enumerate(df.itertuples()):
    sparse_dataset[i, customer[j.User]] = 1
    sparse_dataset[i, col1 + movie[j.Movie]] = 1

Wall time: 9min 45s


In [11]:
sparse_dataset = sc.sparse.csr_matrix(sparse_dataset)

In [2]:
sc.sparse.save_npz(r'D:\Bugaychenko\Netflix\sparse_dataset.npz', sparse_dataset)

NameError: name 'sparse_dataset' is not defined

# Загрузка спарс сета


In [2]:
sparse_dataset = sc.sparse.load_npz(r'D:\Bugaychenko\Netflix\sparse_dataset.npz')
y = np.load('target.npy').reshape(-1, 1)

### FM

In [3]:
def lr_decrease(lr,lr_dec,epochs):
    lr = lr * np.exp(lr_dec*epochs)
    return lr

def Nonlinear(X):  
    #1/2 (X*V)^2 - X^2*V^2
    return 1/2 * np.sum(
        np.power(X @ V, 2) - X.power(2) @ np.power(V, 2),axis=1, keepdims=True)
        
def predict(X):
    return w0 + X @ w1 + Nonlinear(X)
    
def fit(X, Y, epochs=5 , batch_size = 100, lr=0.001, lr_dec=0.001, d=5):
    global w0 
    global w1 
    global V

    w0 = 0
    w1 = np.ones((X.shape[1], 1))
    V = np.ones((X.shape[1], d))
    n = X.shape[0]
    Z = None
    
    for epoch in range(0, epochs):
        batch_indices = np.random.choice(n, batch_size)
        x = X[batch_indices, :]
        y = Y[batch_indices]
        y_hat = predict(x)
        dy = 2 * (y_hat - y) # производная mse
        w0 -= np.mean(dy) * lr 
        w1 -= x.multiply(dy).mean(axis=0).T * lr
        Z = x @ V
        for i in range(d):
            left = x.multiply(Z[:, i].reshape(-1, 1))
            right = x.power(2).multiply(V[:, i])
            V[:, i] -= (np.asarray((left - right).multiply(dy).mean(axis=0).T) * lr).flatten() 
        lr = lr_decrease(lr,lr_dec,epochs)

In [23]:
%%time

rmse_train = []
rmse_test = []
kfold = KFold(n_splits=5)
count = 0


#from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(sparse_dataset, y, test_size=0.4, random_state=2)
# FM2 = fit(X_train, y_train,epochs=16, batch_size=1000, lr=0.3, lr_dec=0.002, d=2)
# print(np.sqrt(sk.metrics.mean_squared_error(y_test, predict(X_test))))
# print(np.sqrt(sk.metrics.mean_squared_error(y_train, predict(X_train))))


for indexTrain, indexTest in kfold.split(sparse_dataset):
    
    X_train = sparse_dataset[indexTrain]
    y_train = y[indexTrain]
    
    X_test = sparse_dataset[indexTest]
    y_test = y[indexTest]
    FM2 = fit(X_train, y_train,epochs=16, batch_size=1000, lr=0.3, lr_dec=0.003, d=5)

    rmse_test.append(np.sqrt(sk.metrics.mean_squared_error(y_test, predict(X_test))))
    rmse_train.append(np.sqrt(sk.metrics.mean_squared_error(y_train, predict(X_train))))
    
    count += 1
    print('{} fold'.format(count))

1 fold
2 fold
3 fold
4 fold
5 fold
Wall time: 1min 7s


In [20]:
df = pd.DataFrame(np.vstack([rmse_train, rmse_test]), index=[
    'rmse_train',
    'rmse_test',
])

df = pd.concat([df, df.mean(axis=1).rename('mean'), df.std(axis=1).rename('std')], axis=1)

df

,0,1,2,3,4,mean,std
rmse_train,1.074544,1.073932,1.073095,1.070737,1.071435,1.072749,0.001622
rmse_test,1.073970,1.078387,1.090617,1.100161,1.094613,1.087550,0.011031
